# Tree Segmentation Example
This is a simple example of what the PyCrown package can do: from pre-calculated rasters (CHM, DSM and DTM) and a height-normalized 3D LiDAR point cloud, individual trees can be segmented.
Outputs are shapefiles of tree top locations, crown shapes as well as a .LAS-file containing classified trees.

## Start with importing the modules

In [1]:
import sys
from datetime import datetime
from pycrown import PyCrown

## Set input files
Specify the file locations for the CHM, DSM, DTM and the LiDAR point cloud.
The latter is only needed if the point cloud should be classified into individual trees.

In [2]:
F_CHM = 'chm.tif'
F_DTM = 'dtm.tif'
F_DSM = 'dsm.tif'
F_LAS = 'clf_norm_merge.las'

## Initialize an instance of PyCrown

In [3]:
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result_9m')

## Clip all input data to new bounding box.

In [4]:
#PC.clip_data_to_bbox((1802150, 1802408, 5467305, 5467480))

## Smooth CHM
A 5x5m block median filter is used (set circular=True to enable a disc-shaped window).

In [5]:
help(PC.filter_chm)

Help on method filter_chm in module pycrown.pycrown:

filter_chm(ws, ws_in_pixels=False, circular=False) method of pycrown.pycrown.PyCrown instance
    Pre-process the canopy height model (smoothing and outlier removal).
    The original CHM (self.chm0) is not overwritten, but a new one is
    stored (self.chm).
    
    Parameters
    ----------
    ws :            int
                    window size of smoothing filter in metre (set in_pixel=True, otherwise)
    ws_in_pixels :  bool, optional
                    sets ws in pixel
    circular :      bool, optional
                    set to True for disc-shaped filter kernel, block otherwise



In [6]:
PC.filter_chm(5, ws_in_pixels=True, circular=False)

## Tree Detection with local maximum filter

In [7]:
help(PC.tree_detection)

Help on method tree_detection in module pycrown.pycrown:

tree_detection(raster, resolution=None, ws=5, hmin=20, return_trees=False, ws_in_pixels=False) method of pycrown.pycrown.PyCrown instance
    Detect individual trees from CHM raster based on a maximum filter.
    Identified trees are either stores as list in the tree dataframe or
    returned as ndarray.
    
    Parameters
    ----------
    raster :        ndarray
                    raster of height values (e.g., CHM)
    resolution :    int, optional
                    resolution of raster in m
    ws :            float
                    moving window size (in metre) to detect the local maxima
    hmin :          float
                    Minimum height of a tree. Threshold below which a pixel
                    or a point cannot be a local maxima
    return_trees :  bool
                    set to True if detected trees shopuld be returned as
                    ndarray instead of being stored in tree dataframe
    ws_i

In [8]:
PC.tree_detection(PC.chm, ws=9*10, hmin=16., ws_in_pixels=True)

## Clip trees to bounding box 
(no trees on image edge)
original extent: 1802140, 1802418, 5467295, 5467490    

In [9]:
#PC.clip_trees_to_bbox(bbox=(1802160, 1802400, 5467315, 5467470))

## Crown Delineation

In [10]:
PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=15.,
                     th_seed=0.7, th_crown=0.55, max_crown=10.)

Tree crowns delineation: 0.184s


## (Optional) Correct tree tops on steep terrain

In [11]:
PC.correct_tree_tops()

Number of trees: 239
Tree tops corrected: 23
Tree tops corrected: 9.623430962343097%
DSM correction: 15
COM correction: 8


(15, 8)

## Calculate tree height and elevation

In [12]:
PC.get_tree_height_elevation(loc='top')
PC.get_tree_height_elevation(loc='top_cor')

## Screen small trees

In [13]:
#PC.screen_small_trees(hmin=20., loc='top')

## Convert raster crowns to polygons

In [14]:
PC.crowns_to_polys_raster()
PC.crowns_to_polys_smooth(store_las=True)

Converting LAS point cloud to shapely points
Converting raster crowns to shapely polygons
Attach LiDAR points to corresponding crowns
Create convex hull around first return points
Classifying point cloud


## Check that all geometries are valid

In [15]:
PC.quality_control()

## Print out number of trees

In [16]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 239


## Export results

In [17]:
PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')
PC.export_tree_locations(loc='top')
PC.export_tree_locations(loc='top_cor')
PC.export_tree_crowns(crowntype='crown_poly_raster')
PC.export_tree_crowns(crowntype='crown_poly_smooth')